# Lending Clug Case Study: Credit Risk
description...

## Imports & Settings

In [1]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

pd.options.plotting.backend = "plotly"


In [2]:
# for better visibility, we will display the whole dataframe and show floats with 2 decimal places
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# set the renderer to display plots in VSCode
pio.renderers.default = "notebook_connected"


## First Look

In [3]:
df = pd.read_csv('loan.csv')
df.set_index('id', inplace=True)
df.head(5)


/Users/houghton/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.



,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975.00,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.00,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0.00,0.00,5863.16,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1077430,1314167,2500,2500,2500.00,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.00,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1077175,1313524,2400,2400,2400.00,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.00,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0.00,0.00,3005.67,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1076863,1277178,10000,10000,10000.00,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.00,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,Feb-96,1,35.00,NaN,10,0,5598,21%,37,f,0.00,0.00,12231.89,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,N

## Data Dictionary

In [4]:
# For easy access we can create a dataframe with the descriptions from the accompanying Data_Dictionary.xlsx

data_dict = pd.read_excel('Data_Dictionary.xlsx',
                          index_col=0, header=None).iloc[:, 0:1].transpose()

# the last two columns don't contain descriptions, so we will drop them
data_dict = data_dict.drop(columns=np.nan)
data_dict


KeyError: '[nan] not found in axis'

## Data Clean Up

While importing, Pandas already warns us that there are columns with mixed data types. 
> 'DtypeWarning: Columns (47) have mixed types.'

For the analysis we need our data to be correctly formatted (preferably numeric).

### Empty & Monotonous Columns
If the data contains completely empty columns we can drop them as they don't contain any information to analyze.

In [ ]:
empty_cols = [col for col in df.columns if df[col].isnull().all()]
print(len(empty_cols), 'empty columns found')
print('Empty columns:', empty_cols)

df.drop(empty_cols, axis=1, inplace=True)


In [ ]:
# reviewing distribution of null values in columns

col_null_values = df.isnull().sum()
col_null_values.hist(nbins=20)


Columns that contain nothing but one value don't add any information and can be droped. We also look at columns with 2 values to see if they can be dropped.

In [ ]:
# dropping columns with single unique values
cols_with_1_value = [col for col in df.columns if df[col].nunique() == 1]
df.drop(cols_with_1_value, axis=1, inplace=True)


In [ ]:
# dropping columns with two unique values
cols_with_2_values = [col for col in df.columns if df[col].nunique() == 2]
for col in cols_with_2_values:
    print(col, df[col].unique())


In [ ]:
df.next_pymnt_d.value_counts()


Just the 'term' (duration of the loan) (36 or 60 months) and the 'next_pymnt_d' (June or July 2016 if any) have only 2 different values. 
<br>Duration might be a factor associated with credit risk, so we leave that column in.
<br>We assume that the cut-off date was in or before June 2016, so that the entries in 'next_pymnt_d' were in the future when the data was taken. This would explain why there are so few and it fits with the time frame of the project description 
(2011 + 60 months = 2017). We don't know if these payments happened but might need the information and therefor leave the column in.

In [ ]:
# Since we are interested in finding predictors for default risk a.k.a. the 'loan_status' column, we can drop the currently active contracts as we don't know their outcome.

df = df[df['loan_status'] != 'Current']

# We can also check for null values in the loan status column.
df['loan_status'].isnull().sum()


### Units

We can convert units that are spelled out in each cell to numeric values

In [ ]:
# converting all columns with units in in values to numbers
df['term'] = df['term'].str.replace(' months', '').astype(int)
df.rename(columns={'term': 'term_in_months'}, inplace=True)

df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)
df['revol_util'] = df['revol_util'].str.replace('%', '').astype(float)


In [ ]:
df.emp_length.unique()


We see above that the employment length has 3 vaules (<1,1 and 10+) that don't end in 'years'. We can convert these.


In [ ]:
df['emp_length'] = df['emp_length'].replace('10+ years', 10).astype(str)
df['emp_length'] = df['emp_length'].replace('< 1 year', 0.5).astype(str)
df['emp_length'] = df['emp_length'].replace('1 year',     1).astype(str)

df['emp_length'] = df['emp_length'].str.replace(' years', '').astype(float)


### Dates

In [ ]:
# Looking at the table we can see the following columns contain dates
cols_with_date = [
    'issue_d', 'earliest_cr_line',
    'last_pymnt_d', 'next_pymnt_d',
    'last_credit_pull_d'
]

for col in cols_with_date:
    df[col] = pd.to_datetime(df[col], format='%b-%y')


### Zip Codes
We can use the provided first 3 digits of ZIP codes to identify any (rough) geographic pattern.

In [ ]:
df.zip_code = df.zip_code.str.replace('xx', '')
df.zip_code.value_counts()

### Description Column

The description column ('desc') contains unstructured data. Let's see if we can extract usefull information from it.

In [ ]:
print(len(df.desc.unique()), 'unique descriptions')

print('Examples:')
for desc in df.sample(5).desc:
    print('-', str(desc).strip())


The descriptions often contain the date when the borrower was added and their usecase. Although the usecase might be interesting for further analysis we only extract the date so we can use it in the bivariate analysis later.

In [ ]:
# extract the date from the description if the description starts with 'Borrower added on'
def extract_added_date(desc):
    desc = str(desc).strip()
    if desc.startswith('Borrower added on'):
        return desc.split(' >')[0][-8:]
    else:
        return None


# insert the new column next to the description column and convert it to datetime
df.insert(df.columns.get_loc('desc') + 1, 'borrower_added_d',
          df.desc.apply(extract_added_date))
df['borrower_added_d'] = pd.to_datetime(
    df['borrower_added_d'], format='%m/%d/%y', errors='coerce')


In [ ]:
# The web addresses of the URL column are inaccessible and therefore beyond the scope of this assignment.
df.drop(['url'], axis=1, inplace=True)


### Grades

In [ ]:
# Converting the sub_grade scale to categorical numerical values

df['grade_num'] = df.sub_grade.astype("category").cat.codes
df['grade_num'].value_counts()

# We can drop the grade and sub_grade column as it is no longer needed.
df.drop(['sub_grade', 'grade'], axis=1, inplace=True)

### Excluding with Data Dict
With the description of the column content we can dop more columns


In [ ]:
# Create one set with column names that are in both the DF and Data Dictionary

cols_from_both = list(set(df.columns).intersection(set(data_dict.columns)))


pd.set_option('display.max_colwidth', None)

#Display the columns and their descriptions from the Data Dictionary
data_dict.transpose().loc[cols_from_both].style.set_properties(
    **{'text-align': 'left'})


In [ ]:
pd.reset_option('display.max_colwidth')


From the displayed list of descriptions above we can exclude some of the columns that are irrelevant to the analysis.

In [ ]:
# As we try to predict the loan status before the loan is given, we can drop the columns that show information about events occurring during the loan period or after the customer has already defaulted.
cols_to_drop = ['collection_recovery_fee','total_rec_late_fee', 'last_pymnt_d']

# We excluded ongoing loans and can therefore drop the following columns
cols_to_drop = cols_to_drop + ['out_prncp','total_pymnt','out_prncp_inv','total_rec_int','last_pymnt_amnt','next_pymnt_d', 'total_rec_prncp', 'funded_amnt_inv']


print(f'These {len(cols_to_drop)} columns will be dropped:')
print(cols_to_drop)

In [ ]:
df.drop(cols_to_drop, axis=1, inplace=True)

## Derived Metrics

### Income to interest rate
We can evaluate if the ratio of income to interest rate had any bearing on loan_status

In [ ]:
df['annual_inc'].plot.box()

It's evident that there are some outliers in the income featureset. We can drop rows that are outside the upper fence.

In [ ]:
df.drop(df.index[df['annual_inc'] > 200000], inplace=True)

In [ ]:
df['annual_inc'].plot.box()

The distribution looks much cleaner now. Now we're ready to create a new derived metric to understand the relationship between and expertise (a combination between income and employment length) an loan_status.

In [ ]:
df['quality_of_expertise'] = df['annual_inc'] / df['emp_length']

In [ ]:
# We'll go ahead and do min-max normalization on the quality of expertise column to allow us us to get a better read on this variable over time.
df['quality_of_expertise_norm'] =(df['quality_of_expertise']-df['quality_of_expertise'].min())/(df['quality_of_expertise'].max()-df['quality_of_expertise'].min())

In [ ]:
fig = go.Figure(data=go.Scatter(x=df[df['loan_status']== 'Fully Paid']['loan_status'],
                                y=df['quality_of_expertise_norm'],
                                mode='markers',
                                marker=dict(
                                    size=2,
                                    color=np.random.randn(500), #set color equal to a variable
                                    colorscale='Viridis', # one of plotly colorscales
                                    showscale=True
    )))

fig.show()

We can see from the graph above that the concetration of 'Charged Off' accounts lessons as the quality of expertise goes up.

In [ ]:
df['total_acc'].mean()

In [ ]:
df['annual_inc'].mean()

In [ ]:
df.hist(x='issue_d' ,y= 'int_rate', nbins=100)

## Overview

In [ ]:
df.info()


In [ ]:
df.describe()


Now that we've done some cleaning up of the data, we can begin to explore and understand different feature at length and its surface level impact to loan _status. We'll start with features we feel could be valuable indicators of correlation to loan_status.

## Feature Analysis

### Function Definitions

In [ ]:
def plot_distribution(column_name):
    """ Plots feature value distribution in both Violin and Box plot formats. """

    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(
        go.Violin(x=df[column_name], name="Violin plot"),
        row=1, col=1
    )

    fig.add_trace(
        go.Box(x=df[column_name], name="Box plot"),
        row=2, col=1
    )

    fig.update_layout(height=600, width=800,
                      title_text=f"Distribution of '{column_name}' values")
    fig.show()

    return


In [ ]:
def plot_surface_impact(column_name):
    """ Plots loan_status' impact to N feature's mean, median, max and min values. """

    fig = make_subplots(rows=4, cols=1)

    variable_impact = pd.DataFrame()
    variable_impact['mean_impact'] = df.groupby(['loan_status'])[
        column_name].mean()
    variable_impact['median_impact'] = df.groupby(
        ['loan_status'])[column_name].median()
    variable_impact['max_impact'] = df.groupby(['loan_status'])[
        column_name].max()
    variable_impact['min_impact'] = df.groupby(['loan_status'])[
        column_name].min()

    fig.add_trace(
        go.Bar(y=variable_impact['mean_impact'],
               x=variable_impact.index, name="Impact to mean"),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(y=variable_impact['median_impact'],
               x=variable_impact.index, name="Impact to median"),
        row=2, col=1
    )

    fig.add_trace(
        go.Bar(y=variable_impact['max_impact'],
               x=variable_impact.index, name="Impact to max"),
        row=3, col=1
    )

    fig.add_trace(
        go.Bar(y=variable_impact['min_impact'],
               x=variable_impact.index, name="Impact to min"),
        row=4, col=1
    )

    fig.update_layout(height=600, width=800,
                      title_text=f"Impact of '{column_name}' on surface statistics of 'loan_status' values")
    fig.show()

    return


In [ ]:
def get_default_rate(col_name, min_count=30):
    """ Returns the rate at which customers that defaulted on the loan for each unique value in the passed column. Minimum Count is the minimum number of customers that must have the value in order to be included in the calculation. """

    default_rate = {}

    for val in df[col_name].unique():

        total = len(df.loc[df[col_name] == val])
        if total >= min_count:
            defaults = len(df.loc[(df[col_name] == val) & (df['loan_status'] == 'Charged Off')])
            default_rate[val] = defaults / total

    return default_rate


In [ ]:
def plot_from_dict(dict):
    """ Plots a bar chart from a dictionary. """

    df = pd.DataFrame.from_dict(dict, orient='index')
    df.sort_values(by=0, inplace=True, ascending=False)
    plt = df.plot(kind='bar')
    plt.show()

### Feature Plots

To visualize the data we can plot a number of selected variables to get impression of their distribution.

In [ ]:
plot_distribution('loan_amnt')
plot_surface_impact('loan_amnt')

plot_distribution('funded_amnt')
plot_surface_impact('funded_amnt')

plot_distribution('dti')
plot_surface_impact('dti')

plot_surface_impact('term_in_months')

plot_distribution('int_rate')
plot_surface_impact('int_rate')


### Summary of initial distribution findings

- The mean and min loan amount for accounts "Charged Off" are slightly higher than accounts that were 'Fully Paid'
- The debt to income ratio is higher for accounts that were 'Charged Off' than accounts that were 'Fully Paid'
- The loan term was several months longer on average for 'Charged Off' accounts than loan terms for 'Fully Paid' accounts.
- Mean interest rates were higher for 'Charged Off' accounts than 'Fully Paid' accounts.
- As expected, the mean and max late fee values for 'Charged Off' accounts were much higher than 'Fully Paid' accounts.

Secondary lessons:
- While the mean may have shifted the max and min values remain rather constant. In order to account for this skewed distribution in our analysis, we also looked at the median of the feature, illustrated in the red bar charts above.
- Debt to income and interest rate appear to have slightly higher median values for 'Charged Off' accounts compared to 'Fully Paid' accounts.

### Correlation Matrix

In [ ]:
df.head()


In [ ]:
df['loan_status_num'] = df.loan_status.astype("category").cat.codes


In [ ]:

# Generate correlation matrix
df_corr = df.corr() 


# Plot heatmap of correlation matrix
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x=df_corr.columns,
        y=df_corr.columns,
        z=np.array(df_corr)
    )
)
fig.update_layout(height=1000, width=1000)
fig.show()


## Geo Data

In [ ]:
default_rate_by_state = get_default_rate('addr_state')
default_rate_by_zip = get_default_rate('zip_code')

In [ ]:
plot_from_dict(default_rate_by_state)

While we do see a gradual distribution of interest rates by states, its worthwhile to note that the delta between Nevada and South Dakota is much larger than the delta between other US states. When evaluating loan applications, predicting 'Charge Offs before they happen will most likely need to take into consideration state or region.

In [ ]:
plot_from_dict(default_rate_by_zip)

## Purchases
We can check if the purpose of the loan has any bearing on the default rate.

In [ ]:
default_rate_by_purpose = get_default_rate('purpose')

plot_from_dict(default_rate_by_purpose)

For the reportet spending categories we see a significant higher default rate for small businesses. Because we know that interest rates were higher for accounts that ultimately had a status of 'Charged Off', this would present as a feature of interest for lending companies looking to be more cautious with their loan approvals.

### Next steps
- ☑️ Purchases analysis
- ☑️ Zip code analysis
- ☑️ Reduce number of features that are not needed (post 'Charge Off' status)
- Derived metrics (description)
- Normalize continuous variables



### Last steps
- Creation of slides